# Tabel of content

1. [Считывание и склейка данных](#first-bullet)
2. [Модель longformer](#second-bullet)
3. [Модель rut5_multitask](#third-bullet)
4. [Модель fredt5_ru](#fourth-bullet)
5. [Модель vikr](#fifth-bullet)
6. [Модель starling](#six-bullet)
7. [Модель use-cmlm](#seven-bullet)
8. [Модель paraphrase](#eight-bullet)

# Считывание и склейка данных <a class="anchor" id="first-bullet"></a>

In [53]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import re
import warnings

warnings.filterwarnings('ignore')

In [25]:
people_df = pd.read_csv('Медицинские_Протоколы_за_сентябрь_2023.csv', sep=';')

In [26]:
people_df.head()

,MedicalRecordKey,MedicalRecordDate,PatientKey,Возраст,Пол,СпециальностьВрача,Жалобы,ПеренесенныеЗаболевания,ПеренесенныеОперации,ПринимаемыеПрепараты
0,106601974,2023-09-01 11:43:56.000,0x9F983DD5D8714DDD306357EE4D64452AD799C6393D17...,1892.0,0.0,Врач педиатр,неустойчивый стул. шкала ВАШ -0. шкала Хампти-...,"Факт получения травм, ожогов, ран, проведения ...",Не было.,Не отягощен
1,106601996,2023-09-01 11:44:14.000,0x10464D4CDD0C380C6305D01418D3C55EEE4D47FD4784...,10893.0,1.0,Врач гастроэнтеролог,"на рвоту 2 раза в день, тошноту, общую слабост...","Факт получения травм, ожогов, ран, проведения ...",NaN,NaN
2,106601997,2023-09-01 11:44:09.000,0xCD6CF235D41371DD1E46CA4B01342381B56CCB51B1AD...,9702.0,0.0,Врач терапевт,"- гнусавость голоса, заложенность носа перед с...","Факт получения травм, ожогов, ран, проведения ...",NaN,NaN
3,106601998,2023-09-01 11:44:18.000,0x256CCD8B1BCF4F03C66EAABEEC017C7109806AA4CEFD...,5402.0,1.0,Врач невролог,последние полгода головные боли давящего харак...,"Факт получения травм, ожогов, ран, проведения ...",NaN,NaN
4,106602005,2023-09-01 11:43:43.000,0xCC1D2E1557D37A1E9071BA7B4DE4477FA27C902CCE69...,14306.0,1.0,дермат,"На эпизодически возникающий дискомфорт, нагру...","Факт получения травм, ожогов, ран, проведения ...","по поводу кисты яичника, внематочная беременн...",не отягощен


In [27]:
people_df.shape

(415421, 10)

In [28]:
diag = pd.read_csv('Диагнозы_за_сентябрь_2023.csv', sep = ';')

In [29]:
diag.shape

(664743, 4)

In [30]:
diag.head()

,MedicalRecordKey,N_LINE,КодМКБ,ДиагнозПоМКБ
0,106599178,1,S06.00,Сотрясение головного мозга без открытой внутри...
1,106599178,2,S69.9,Травма запястья и кисти неуточненная|Другие и ...
2,106599251,1,S86.1,Травма другой (их) мышцы (мышц) и сухожилия (и...
3,106599253,1,J03.9,Острый тонзиллит неуточненный|Острый тонзиллит...
4,106599267,1,N41.1,Хронический простатит|Воспалительные болезни п...


In [31]:
diag = diag.merge(people_df, on = 'MedicalRecordKey', how = 'left')

In [32]:
articles = pd.read_csv('data.csv')

In [33]:
articles.head()

,Unnamed: 0,title,text,subtitles,subtitle
0,0,Интравитреальные инъекции,Интравитреальная инъекция – процедура введения...,NaN,"Когда принято проводить такую процедуру?, Виды..."
1,1,Диабетическая ретинопатия,"Сахарный диабет – заболевание, от которого сег...",NaN,"Диабетическая ретинопатия: что это такое, поче..."
2,2,Гимнастика для глаз при близорукости,Различные тренировки зрения при близорукости с...,NaN,"Как связаны близорукость и глазные мышцы?, Чем..."
3,3,"Гемофтальм: причины, симптомы, диагностика",Гемофтальм – это кровоизлияние в полость стекл...,NaN,"Причины гемофтальма, Признаки гемофтальма"
4,4,Вторичная глаукома,По данным ВОЗ (Всемирной организации здравоохр...,NaN,"Вторичная глаукома: что это?, Основные формы в..."


In [34]:
diag.head()

,MedicalRecordKey,N_LINE,КодМКБ,ДиагнозПоМКБ,MedicalRecordDate,PatientKey,Возраст,Пол,СпециальностьВрача,Жалобы,ПеренесенныеЗаболевания,ПеренесенныеОперации,ПринимаемыеПрепараты
0,106599178,1,S06.00,Сотрясение головного мозга без открытой внутри...,2023-09-01 11:03:46.000,0x9529545E3E199F69FCC7FC5D91EA7811A818FC7B4ADB...,11627.0,0.0,Врач невролог,"Головная боль в области ссадин, светобоязнь, т...","Факт получения травм, ожогов, ран, проведения ...",Отрицает,NaN
1,106599178,2,S69.9,Травма запястья и кисти неуточненная|Другие и ...,2023-09-01 11:03:46.000,0x9529545E3E199F69FCC7FC5D91EA7811A818FC7B4ADB...,11627.0,0.0,Врач невролог,"Головная боль в области ссадин, светобоязнь, т...","Факт получения травм, ожогов, ран, проведения ...",Отрицает,NaN
2,106599251,1,S86.1,Травма другой (их) мышцы (мышц) и сухожилия (и...,2023-09-01 11:04:46.000,0x5C9681F364FCE1E5F3BC8688C373BD2AF512215E842E...,15578.0,0.0,Врач травматолог ортопед,на боли в области правой голени при движениях ...,"Факт получения травм, ожогов, ран, проведения ...",Аппендэктомия.,Базисной терапии нет.
3,106599253,1,J03.9,Острый тонзиллит неуточненный|Острый тонзиллит...,2023-09-01 11:04:53.000,0x18B0F813163D92FD222813D4F5D2B7EE606CAC6F01CF...,8821.0,1.0,Врач терапевт,На момент осмотра предъявляет жалобы на повыше...,"Факт получения травм, ожогов, ран, проведения ...",NaN,NaN
4,106599267,1,N41.1,Хронический простатит|Воспалительные болезни п...,2023-09-01 11:04:59.000,0x3208D40A66F2E932A43BD4619BEBCD3908292864F5CD...,18583.0,0.0,Врач уролог,"-***-* по ДМС на учащенные мочеиспускания,...",отрицает ОРВИ хр простатит,отрицает,NaN


In [35]:
diag.dropna(subset=['ДиагнозПоМКБ'], inplace=True)
diag.reset_index(inplace=True)

In [36]:
import warnings
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

In [37]:
articles.dropna(subset=['subtitle'], inplace=True)
articles.reset_index(inplace=True)

In [38]:
for j in tqdm(range(articles.shape[0])):
    stmp_str = articles['title'][j].split()
    for i in range(len(stmp_str)):
        stmp_str[i] = stmp_str[i].replace('|', ' ')
        stmp_str[i] = [w for w in stmp_str[i] if (w.isalpha() or w==' ' or w in '.,?!:;')]
        stmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), stmp_str[i]))
        stmp_str[i] = ''.join(stmp_str[i])
    articles['title'][j] = ' '.join(stmp_str)
    articles['title'][j] = ' '.join(articles['title'][j].split())

  0%|          | 0/553 [00:00<?, ?it/s]

In [39]:
for j in tqdm(range(articles.shape[0])):
    stmp_str = articles['subtitle'][j].split()
    for i in range(len(stmp_str)):
        stmp_str[i] = stmp_str[i].replace('|', ' ')
        stmp_str[i] = [w for w in stmp_str[i] if (w.isalpha() or w==' ' or w in '.,?!:;')]
        stmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), stmp_str[i]))
        stmp_str[i] = ''.join(stmp_str[i])
    articles['subtitle'][j] = ' '.join(stmp_str)
    articles['subtitle'][j] = ' '.join(articles['subtitle'][j].split())

  0%|          | 0/553 [00:00<?, ?it/s]

In [40]:
diag['ДиагнозПоМКБ'].isna().sum()

0

In [41]:
diag.columns

Index(['index', 'MedicalRecordKey', 'N_LINE', 'КодМКБ', 'ДиагнозПоМКБ',
       'MedicalRecordDate', 'PatientKey', 'Возраст', 'Пол',
       'СпециальностьВрача', 'Жалобы', 'ПеренесенныеЗаболевания',
       'ПеренесенныеОперации', 'ПринимаемыеПрепараты'],
      dtype='object')

In [42]:
diag.dropna(subset=['Жалобы'], inplace= True)

# Модель longformer <a class="anchor" id="second-bullet"></a>

In [58]:
import torch
from transformers import LongformerModel, LongformerTokenizerFast
import numpy
from sklearn.metrics.pairwise import cosine_similarity
import torch.nn.functional as F
# Загрузка предобученной модели и токенизатора
model = LongformerModel.from_pretrained('kazzand/ru-longformer-tiny-16384')
tokenizer = LongformerTokenizerFast.from_pretrained('kazzand/ru-longformer-tiny-16384')


def longfromer_pred(diag_str):

    similarities = []
    tmp_str = diag_str.split()
    for i in range(len(tmp_str)):
        tmp_str[i] = tmp_str[i].replace('|', ' ')
        tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w==' ')]
        tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
        tmp_str[i] = ''.join(tmp_str[i])
    
    cur_article = numpy.NaN
    sim = 0 
    for i in range(articles.shape[0]):
        cur_diag = ' '.join(tmp_str)
        cur_diag = ' '.join(cur_diag.split())
        top_50 = ' '.join(articles['subtitle'][i].split()[:50])
        art_text = ' '.join(top_50.split())
        art_text = articles['title'][i]
        if len(cur_diag)>len(art_text):
            art_text = art_text+' '*(len(cur_diag)-len(art_text))
            
        else:
            cur_diag = cur_diag+' '*(len(art_text)-len(cur_diag))
    
        
        encoding1 = tokenizer.encode(cur_diag, truncation=True)
        encoding1 = numpy.array(encoding1)
        
        encoding2 = tokenizer.encode(art_text,truncation=True)
        encoding2 = numpy.array(encoding2)
        if encoding1.shape[0]>encoding2.shape[0]:
            zero_vector = numpy.zeros((1, len(encoding1)-len(encoding2)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            #encoding2 = encoding2.reshape(len(encoding2),1)
            #encoding2 = numpy.concatenate((encoding2, zero_vector))
        else:
            zero_vector = numpy.zeros((1, len(encoding2)-len(encoding1)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            #encoding1 = encoding1.reshape(len(encoding1),1)
            #encoding1 = numpy.concatenate((encoding1, zero_vector))
        inputs_1 = torch.tensor([encoding1]).unsqueeze(0)
        inputs_2 = torch.tensor([encoding2]).unsqueeze(0)
        with torch.no_grad():
            outputs_1 = model(inputs_1[0])
            outputs_2 = model(inputs_2[0])
        similarity = cosine_similarity(outputs_1[0][0][0].reshape(1,-1),outputs_2[0][0][0].reshape(1,-1))
        similarities.append(similarity)
    similarities = np.array(similarities)
    ind = np.argpartition(similarities.reshape(similarities.shape[1], similarities.shape[0]), -4)[:, -4:]
    values = similarities[ind]
    return ind, values

Some weights of LongformerModel were not initialized from the model checkpoint at kazzand/ru-longformer-tiny-16384 and are newly initialized: ['longformer.pooler.dense.bias', 'longformer.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
ind, values = longfromer_pred(diag['Жалобы'][0])

In [66]:
print("Жалоба: ",diag['Жалобы'][0])
print("Рекомендация: ",articles['title'][ind[0][0]],'\nБлизость: ',values[0][0],"\n")
print("Рекомендация: ", articles['title'][ind[0][1]],"\nБлизость: ",values[0][1],"\n")
print("Рекомендация: ",articles['title'][ind[0][2]],'\nБлизость: ',values[0][2],"\n")
print("Рекомендация: ",articles['title'][ind[0][3]],'\nБлизость: ',values[0][3],"\n")

Жалоба:  Головная боль в области ссадин, светобоязнь, тошнота, боль в левой руке, в области ссадин на коленях, на правой руке.
Рекомендация:  Реабилитация после травм спины спинного мозга 
Близость:  [[0.6644475]] 

Рекомендация:  Мезотерапия вокруг глаз как спасение от мешков, отёков и тёмных кругов 
Близость:  [[0.6717312]] 

Рекомендация:  Лечение варусной деформации нижних конечностей 
Близость:  [[0.7226739]] 

Рекомендация:  Лечение вальгусной деформации стопы 
Близость:  [[0.70205754]] 



# Модель rut5-multitask <a class="anchor" id="third-bullet"></a>

In [69]:
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import numpy
# Загрузка предобученной модели и токенизатора
tokenizer = T5Tokenizer.from_pretrained("cointegrated/rut5-base-multitask")
model = T5ForConditionalGeneration.from_pretrained("cointegrated/rut5-base-multitask")

def rut5_pred(diag_str):
    similarities = []
    cur_article = numpy.NaN
    sim = 0 
    tmp_str = diag_str.split()
    for i in range(len(tmp_str)):
        tmp_str[i] = tmp_str[i].replace('|', ' ')
        tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w==' ')]
        tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
        tmp_str[i] = ''.join(tmp_str[i])
        
    for i in range(articles.shape[0]):
        cur_diag = ' '.join(tmp_str)
        cur_diag = ' '.join(cur_diag.split())
        top_50 = ' '.join(articles['subtitle'][i].split())
        art_text = ' '.join(top_50.split())
        art_text = articles['title'][i]
        encoding1 = tokenizer.encode(cur_diag, max_length=512)
        encoding1 = numpy.array(encoding1)
        encoding2 = tokenizer.encode(art_text, max_length=512)
        encoding2 = numpy.array(encoding2)
        if encoding1.shape[0]>encoding2.shape[0]:
            zero_vector = numpy.zeros((1, len(encoding1)-len(encoding2)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding2 = encoding2.reshape(len(encoding2),1)
            encoding2 = numpy.concatenate((encoding2, zero_vector))
        else:
            zero_vector = numpy.zeros((1, len(encoding2)-len(encoding1)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding1 = encoding1.reshape(len(encoding1),1)
            encoding1 = numpy.concatenate((encoding1, zero_vector))
        similarity = cosine_similarity(encoding1.reshape(1,-1),encoding2.reshape(1,-1))
        similarities.append(similarity)
    similarities = np.array(similarities)
    ind = np.argpartition(similarities.reshape(similarities.shape[1], similarities.shape[0]), -4)[:, -4:]
    values = similarities[ind]
    return ind, values

In [70]:
ind, values = rut5_pred(diag['Жалобы'][0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [71]:
print("Жалоба: ",diag['Жалобы'][0])
print("Рекомендация: ",articles['title'][ind[0][0]],'\nБлизость: ',values[0][0],"\n")
print("Рекомендация: ", articles['title'][ind[0][1]],"\nБлизость: ",values[0][1],"\n")
print("Рекомендация: ",articles['title'][ind[0][2]],'\nБлизость: ',values[0][2],"\n")
print("Рекомендация: ",articles['title'][ind[0][3]],'\nБлизость: ',values[0][3],"\n")

Жалоба:  Головная боль в области ссадин, светобоязнь, тошнота, боль в левой руке, в области ссадин на коленях, на правой руке.
Рекомендация:  Проведение трансвагинальной пункции фолликулов, переноса эмбрионов в полость матки, внутриматочной инсеминации 
Близость:  [[0.49563308]] 

Рекомендация:  Остеохондроз что это такое, причины, симптомы, лечение и профилактика. Как лечить остеохондроз позвоночника? 
Близость:  [[0.51552281]] 

Рекомендация:  Биопсия шейки матки нерожавших девушек: показания и противопоказания, влияет ли на способность забеременеть 
Близость:  [[0.6009054]] 

Рекомендация:  Бронхиальная астма: симптомы и лечение, признаки у взрослых 
Близость:  [[0.52806139]] 



# Модель fred_t5_ru <a class="anchor" id="fourth-bullet"></a>

In [74]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "IlyaGusev/fred_t5_ru_turbo_alpaca"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



def fred_t5_alpaca_pred(diag_str):
    similarities = []
    tmp_str = diag_str.split()
    for i in range(len(tmp_str)):
        tmp_str[i] = tmp_str[i].replace('|', ' ')
        tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w==' ')]
        tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
        tmp_str[i] = ''.join(tmp_str[i])
    
    for i in range(articles.shape[0]):
        
        cur_diag = ' '.join(tmp_str)
        cur_diag = ' '.join(cur_diag.split())
        top_50 = ' '.join(articles['subtitle'][i].split()[:25])
        art_text = ' '.join(top_50.split())
        art_text = articles['title'][i]
        encoding1 = tokenizer.encode(cur_diag)
        encoding1 = numpy.array(encoding1)
        encoding2 = tokenizer.encode(art_text, max_length=512)
        encoding2 = numpy.array(encoding2)
        if encoding1.shape[0]>encoding2.shape[0]:
            zero_vector = numpy.zeros((1, len(encoding1)-len(encoding2)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding2 = encoding2.reshape(len(encoding2),1)
            encoding2 = numpy.concatenate((encoding2, zero_vector))
        else:
            zero_vector = numpy.zeros((1, len(encoding2)-len(encoding1)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding1 = encoding1.reshape(len(encoding1),1)
            encoding1 = numpy.concatenate((encoding1, zero_vector))
        similarity = cosine_similarity(encoding1.reshape(1,-1),encoding2.reshape(1,-1))
        similarities.append(similarity)
    similarities = np.array(similarities)
    ind = np.argpartition(similarities.reshape(similarities.shape[1], similarities.shape[0]), -4)[:, -4:]
    values = similarities[ind]
    return ind, values

In [75]:
ind, values = fred_t5_alpaca_pred(diag['Жалобы'][0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [76]:
print("Жалоба: ",diag['Жалобы'][0])
print("Рекомендация: ",articles['title'][ind[0][0]],'\nБлизость: ',values[0][0],"\n")
print("Рекомендация: ", articles['title'][ind[0][1]],"\nБлизость: ",values[0][1],"\n")
print("Рекомендация: ",articles['title'][ind[0][2]],'\nБлизость: ',values[0][2],"\n")
print("Рекомендация: ",articles['title'][ind[0][3]],'\nБлизость: ',values[0][3],"\n")

Жалоба:  Головная боль в области ссадин, светобоязнь, тошнота, боль в левой руке, в области ссадин на коленях, на правой руке.
Рекомендация:  Противопоказания для вакцинации 
Близость:  [[0.59743152]] 

Рекомендация:  Селективный перенос одного эмбриона 
Близость:  [[0.60868135]] 

Рекомендация:  Возможна ли естественная беременность после ЭКО 
Близость:  [[0.61913053]] 

Рекомендация:  Интравитреальные инъекции 
Близость:  [[0.62020584]] 



# Модель Vikhr <a class="anchor" id="fifth-bullet"></a>

In [77]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-7B-instruct_0.4")

def vikhr_pred(diag_str):
    similarities = []
    tmp_str = diag_str.split()
    for i in range(len(tmp_str)):
        tmp_str[i] = tmp_str[i].replace('|', ' ')
        tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w==' ')]
        tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
        tmp_str[i] = ''.join(tmp_str[i])
    for i in range(articles.shape[0]):
        cur_diag = ' '.join(tmp_str)
        cur_diag = ' '.join(cur_diag.split())
        top_50 = ' '.join(articles['subtitle'][i].split()[:25])
        art_text = ' '.join(top_50.split())
        art_text = articles['title'][i]
        encoding1 = tokenizer.encode(cur_diag, max_length=512)
        encoding1 = numpy.array(encoding1)
        encoding2 = tokenizer.encode(art_text, max_length=512)
        encoding2 = numpy.array(encoding2)
        if encoding1.shape[0]>encoding2.shape[0]:
            zero_vector = numpy.zeros((1, len(encoding1)-len(encoding2)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding2 = encoding2.reshape(len(encoding2),1)
            encoding2 = numpy.concatenate((encoding2, zero_vector))
        else:
            zero_vector = numpy.zeros((1, len(encoding2)-len(encoding1)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding1 = encoding1.reshape(len(encoding1),1)
            encoding1 = numpy.concatenate((encoding1, zero_vector))
        similarity = cosine_similarity(encoding1.reshape(1,-1),encoding2.reshape(1,-1))
        similarities = np.array(similarities)
    ind = np.argpartition(similarities.reshape(similarities.shape[1], similarities.shape[0]), -4)[:, -4:]
    values = similarities[ind]
    return ind, values

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [78]:
ind, values = fred_t5_alpaca_pred(diag['Жалобы'][0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [79]:
print("Жалоба: ",diag['Жалобы'][0])
print("Рекомендация: ",articles['title'][ind[0][0]],'\nБлизость: ',values[0][0],"\n")
print("Рекомендация: ", articles['title'][ind[0][1]],"\nБлизость: ",values[0][1],"\n")
print("Рекомендация: ",articles['title'][ind[0][2]],'\nБлизость: ',values[0][2],"\n")
print("Рекомендация: ",articles['title'][ind[0][3]],'\nБлизость: ',values[0][3],"\n")

Жалоба:  Головная боль в области ссадин, светобоязнь, тошнота, боль в левой руке, в области ссадин на коленях, на правой руке.
Рекомендация:  Что такое АСИТтерапия и как ее проводят. Показания, противопоказания, побочные эффекты 
Близость:  [[0.62031335]] 

Рекомендация:  Врач уролог кто это и что лечит? Все о врачебной специальности уролог 
Близость:  [[0.62778454]] 

Рекомендация:  Гидронефроз почек: степени гидронефроза, признаки заболевания, диагностика и методы лечения 
Близость:  [[0.63562919]] 

Рекомендация:  Как подготовиться к беременности: с чего начать, какие анализы необходимо сдавать, организация правильного питания, влияние вредных привычек и другая полезная информация 
Близость:  [[0.6807491]] 



# Модель Starling <a class="anchor" id="six-bullet"></a>

In [80]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained("Nexusflow/Starling-LM-7B-beta")
#model = transformers.AutoModelForCausalLM.from_pretrained("Nexusflow/Starling-LM-7B-beta")

def star_pred(diag_str):
    similarities = []
    tmp_str = diag_str.split()
    for i in range(len(tmp_str)):
        tmp_str[i] = tmp_str[i].replace('|', ' ')
        tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w==' ')]
        tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
        tmp_str[i] = ''.join(tmp_str[i])
    for i in range(articles.shape[0]):
        cur_diag = ' '.join(tmp_str)
        cur_diag = ' '.join(cur_diag.split())
        top_50 = ' '.join(articles['subtitle'][i].split()[:25])
        art_text = ' '.join(top_50.split())
        art_text = articles['title'][i]
        encoding1 = tokenizer.encode(cur_diag, max_length=512)
        encoding1 = numpy.array(encoding1)
        encoding2 = tokenizer.encode(art_text, max_length=512)
        encoding2 = numpy.array(encoding2)
        if encoding1.shape[0]>encoding2.shape[0]:
            zero_vector = numpy.zeros((1, len(encoding1)-len(encoding2)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding2 = encoding2.reshape(len(encoding2),1)
            encoding2 = numpy.concatenate((encoding2, zero_vector))
        else:
            zero_vector = numpy.zeros((1, len(encoding2)-len(encoding1)), dtype=int)
            zero_vector = zero_vector.reshape(len(encoding1)-len(encoding2), 1)
            encoding1 = encoding1.reshape(len(encoding1),1)
            encoding1 = numpy.concatenate((encoding1, zero_vector))
        
        similarity = cosine_similarity(encoding1.reshape(1,-1),encoding2.reshape(1,-1))
        similarities = np.array(similarities)
    ind = np.argpartition(similarities.reshape(similarities.shape[1], similarities.shape[0]), -4)[:, -4:]
    values = similarities[ind]
    return ind, values

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [81]:
ind, values = fred_t5_alpaca_pred(diag['Жалобы'][0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [82]:
print("Жалоба: ",diag['Жалобы'][0])
print("Рекомендация: ",articles['title'][ind[0][0]],'\nБлизость: ',values[0][0],"\n")
print("Рекомендация: ", articles['title'][ind[0][1]],"\nБлизость: ",values[0][1],"\n")
print("Рекомендация: ",articles['title'][ind[0][2]],'\nБлизость: ',values[0][2],"\n")
print("Рекомендация: ",articles['title'][ind[0][3]],'\nБлизость: ',values[0][3],"\n")

Жалоба:  Головная боль в области ссадин, светобоязнь, тошнота, боль в левой руке, в области ссадин на коленях, на правой руке.
Рекомендация:  Близорукость: что это такое, симптомы, методы лечения 
Близость:  [[0.69185162]] 

Рекомендация:  Астигматизм: что это такое, симптомы, методы лечения 
Близость:  [[0.75062663]] 

Рекомендация:  Крапивница: симптомы, причины, виды, лечение и осложнения 
Близость:  [[0.69440774]] 

Рекомендация:  Тонометрия глаза: показания к проведению, виды, методы измерения 
Близость:  [[0.73540849]] 



# Модель use-cmlm <a class="anchor" id="seven-bullet"></a>

In [86]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
#%%
model = SentenceTransformer('sentence-transformers/use-cmlm-multilingual')

cur = diag['Жалобы'][99]

max_match = 0
id_found = 0

tmp_str = cur.split()
for i in range(len(tmp_str)):
    tmp_str[i] = tmp_str[i].replace('|', ' ')
    tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w in'.,?!:;')]
    tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
    tmp_str[i] = ''.join(tmp_str[i])
cur_diag = ' '.join(tmp_str).strip()

article_encoded = model.encode(cur_diag)


counter = 0
art_text = articles['title']
sentences_encoded = model.encode(art_text)
scores = util.semantic_search(article_encoded, sentences_encoded)
cur_max = scores[0][:3]
print('Жалоба: ',cur_diag,'\n')
print('Статьи: ')
print(articles['title'][cur_max[0]['corpus_id']], ' ', cur_max[0]['score'])
print(articles['title'][cur_max[1]['corpus_id']],  cur_max[0]['score'])
print(articles['title'][cur_max[2]['corpus_id']],  cur_max[0]['score'])

Some weights of the model checkpoint at sentence-transformers/use-cmlm-multilingual were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Жалоба:  На остаточный дискомфорт и заложенность правого уха, выраженное першение в горле 

Статьи: 
Шишка на верхнем и нижнем веке глаза: причины заболевания, методы диагностики и лечения   0.4394267201423645
Лечение храпа и синдрома обструктивного апноэ сна 0.4394267201423645
Что делать при боли в деснах 0.4394267201423645


# Модель paraphrase <a class="anchor" id="eight-bullet"></a>

In [87]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
#%%
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

cur = diag['Жалобы'][29]

max_match = 0
id_found = 0

tmp_str = cur.split()
for i in range(len(tmp_str)):
    tmp_str[i] = tmp_str[i].replace('|', ' ')
    tmp_str[i] = [w for w in tmp_str[i] if (w.isalpha() or w in'.,?!:;')]
    tmp_str[i] = list(filter(lambda w: not re.match(r'[a-zA-Z]+', w), tmp_str[i]))
    tmp_str[i] = ''.join(tmp_str[i])
cur_diag = ' '.join(tmp_str).strip()

article_encoded = model.encode(cur_diag)


counter = 0
art_text = articles['title']
sentences_encoded = model.encode(art_text)
scores = util.semantic_search(article_encoded, sentences_encoded)
cur_max = scores[0][:3]
print('Жалоба: ',cur_diag,'\n')
print('Статьи: ')
print(articles['title'][cur_max[0]['corpus_id']], ' ', cur_max[0]['score'])
print(articles['title'][cur_max[1]['corpus_id']],  cur_max[0]['score'])
print(articles['title'][cur_max[2]['corpus_id']],  cur_max[0]['score'])

Жалоба:  Чувство тяжести в области правого подреберья ,после еды. Периодически беспокоят режущие боли в эпигастрии , после еды.  более после употребления фруктов, эпизоды сердцебиений. 

Статьи: 
Синдром раздраженного кишечника: причины и лечение СРК, препараты, диета и профилактика   0.6247788667678833
Блефароспазм причины и симптомы 0.6247788667678833
Крапивница: симптомы, причины, виды, лечение и осложнения 0.6247788667678833
